In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('train_data.csv')
train_data.head()

,id,home_team_abbr,away_team_abbr,date,is_night_game,home_team_win,home_pitcher,away_pitcher,home_team_rest,away_team_rest,...,away_pitcher_H_batters_faced_skew,away_pitcher_BB_batters_faced_mean,away_pitcher_BB_batters_faced_std,away_pitcher_BB_batters_faced_skew,away_pitcher_leverage_index_avg_mean,away_pitcher_leverage_index_avg_std,away_pitcher_leverage_index_avg_skew,away_pitcher_wpa_def_mean,away_pitcher_wpa_def_std,away_pitcher_wpa_def_skew
0,0,KFH,KJP,2021-05-16,False,True,juradar01,carraca01,1.0,1.0,...,1.133350,-0.974559,-0.900633,-1.093425,0.896974,-0.611051,-0.398111,0.949021,1.007072,0.340438
1,1,VJV,HXK,2019-05-04,True,False,ramirer02,rodrich01,2.0,NaN,...,-0.044641,-0.878649,-1.079528,-1.719608,0.050448,-0.851738,-0.202878,0.489511,-0.876286,1.416154
2,2,VJV,JEM,2019-06-10,True,True,jarvibr01,tropeni01,1.0,1.0,...,-0.100180,-1.702937,-0.867762,1.992552,-0.404961,-0.132717,-0.106344,2.481020,-0.200110,-0.026083
3,3,BPH,FBW,2018-06-26,True,True,diazyi01,johnsji04,1.0,1.0,...,-1.385079,-1.549095,-1.008470,0.116080,-1.236753,-0.119898,0.005985,1.646317,-0.764309,NaN
4,4,RLJ,DPS,2016-07-05,True,False,willibr02,armstsh01,1.0,1.0,...,0.356122,0.663967,-0.123547,0.361822,-0.035276,-0.285671,-2.563819,0.527432,-0.911987,-1.109533


In [2]:
import pandas as pd

# Assuming your DataFrame is named df and the features are 'feature1' and 'feature2'
correlation = train_data['home_batting_batting_avg_10RA'].corr(train_data['home_batting_onbase_perc_10RA'])
print("Correlation:", correlation)


Correlation: 0.8603297701624135


In [3]:
# Assuming your DataFrame is named df and the features are 'feature1' and 'feature2'
correlation = train_data['home_batting_batting_avg_10RA'].corr(train_data['home_batting_batting_avg_mean'])
print("Correlation:", correlation)

Correlation: 0.700311503602192


In [4]:
train_data = train_data.applymap(lambda x: float(x) if isinstance(x, bool) else x)

/tmp/ipykernel_15874/2294140884.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_data = train_data.applymap(lambda x: float(x) if isinstance(x, bool) else x)


In [5]:
import pandas as pd

# Function to normalize date features
def normalize_date_to_range(date_series):
    # Convert to datetime
    dates = pd.to_datetime(date_series)
    # Find the min and max dates
    min_date = dates.min()
    max_date = dates.max()
    # Calculate the total range of days
    total_days = (max_date - min_date).days
    # Normalize to range [0, 1]
    normalized = (dates - min_date).dt.days / total_days
    return normalized, min_date, max_date

# Apply the function to your dataset
train_data['date'], min_date, max_date = normalize_date_to_range(train_data['date'])

# Print the min and max dates
print(f"Min date: {min_date}, Max date: {max_date}")
print(train_data.head())


Min date: 2016-04-03 00:00:00, Max date: 2023-07-31 00:00:00
   id home_team_abbr away_team_abbr      date  is_night_game  home_team_win  \
0   0            KFH            KJP  0.698692            0.0            1.0   
1   1            VJV            HXK  0.420935            1.0            0.0   
2   2            VJV            JEM  0.434766            1.0            1.0   
3   3            BPH            FBW  0.304299            1.0            1.0   
4   4            RLJ            DPS  0.034766            1.0            0.0   

  home_pitcher away_pitcher  home_team_rest  away_team_rest  ...  \
0    juradar01    carraca01             1.0             1.0  ...   
1    ramirer02    rodrich01             2.0             NaN  ...   
2    jarvibr01    tropeni01             1.0             1.0  ...   
3     diazyi01    johnsji04             1.0             1.0  ...   
4    willibr02    armstsh01             1.0             1.0  ...   

   away_pitcher_H_batters_faced_skew  away_pitcher_BB_b

In [6]:
# Map unique names to integers for both features without overlap
def map_non_overlapping_features(df, columns):
    # Collect unique names for each column
    mappings = {}
    current_index = 0
    for column in columns:
        unique_names = df[column].unique()
        column_mapping = {name: current_index + idx for idx, name in enumerate(unique_names)}
        mappings[column] = column_mapping
        current_index += len(unique_names)  # Ensure no overlap in indices
        # Apply the mapping
        df[column] = df[column].map(column_mapping)
    return df, mappings

# Apply to 'home_pitcher' and 'away_pitcher'
columns_to_map = ['home_pitcher', 'away_pitcher']
train_data, mappings = map_non_overlapping_features(train_data, columns_to_map)

# Display the result
print(train_data)
print("Mapping dictionaries:")
for column, mapping in mappings.items():
    print(f"{column}: {mapping}")

          id home_team_abbr away_team_abbr      date  is_night_game  \
0          0            KFH            KJP  0.698692            0.0   
1          1            VJV            HXK  0.420935            1.0   
2          2            VJV            JEM  0.434766            1.0   
3          3            BPH            FBW  0.304299            1.0   
4          4            RLJ            DPS  0.034766            1.0   
...      ...            ...            ...       ...            ...   
11062  11062            VQC            MOO  0.035888            1.0   
11063  11063            GKO            VQC  0.961495            0.0   
11064  11064            ECN            QPO  0.839252            1.0   
11065  11065            QDH            HXK  0.699813            1.0   
11066  11066            RAV            HXK  0.973832            1.0   

       home_team_win  home_pitcher  away_pitcher  home_team_rest  \
0                1.0             0           749             1.0   
1          

In [7]:
# Map unique names to integers for both features without overlap
def map_non_overlapping_features(df, columns):
    # Collect unique names for each column
    mappings = {}
    current_index = 0
    for column in columns:
        unique_names = df[column].unique()
        column_mapping = {name: current_index + idx for idx, name in enumerate(unique_names)}
        mappings[column] = column_mapping
        current_index += len(unique_names)  # Ensure no overlap in indices
        # Apply the mapping
        df[column] = df[column].map(column_mapping)
    return df, mappings

# Apply to 'home_pitcher' and 'away_pitcher'
columns_to_map = ['home_team_abbr', 'away_team_abbr']
train_data, mappings = map_non_overlapping_features(train_data, columns_to_map)

# Display the result
print(train_data)
print("Mapping dictionaries:")
for column, mapping in mappings.items():
    print(f"{column}: {mapping}")

          id  home_team_abbr  away_team_abbr      date  is_night_game  \
0          0               0              30  0.698692            0.0   
1          1               1              31  0.420935            1.0   
2          2               1              32  0.434766            1.0   
3          3               2              33  0.304299            1.0   
4          4               3              34  0.034766            1.0   
...      ...             ...             ...       ...            ...   
11062  11062               5              48  0.035888            1.0   
11063  11063              11              39  0.961495            0.0   
11064  11064              29              41  0.839252            1.0   
11065  11065              18              31  0.699813            1.0   
11066  11066              14              31  0.973832            1.0   

       home_team_win  home_pitcher  away_pitcher  home_team_rest  \
0                1.0             0           749       

In [8]:
# Drop non-integer and non-float columns
train_data = train_data.select_dtypes(include=['int64', 'float64'])

# Display the resulting DataFrame
print(train_data)

          id  home_team_abbr  away_team_abbr      date  is_night_game  \
0          0               0              30  0.698692            0.0   
1          1               1              31  0.420935            1.0   
2          2               1              32  0.434766            1.0   
3          3               2              33  0.304299            1.0   
4          4               3              34  0.034766            1.0   
...      ...             ...             ...       ...            ...   
11062  11062               5              48  0.035888            1.0   
11063  11063              11              39  0.961495            0.0   
11064  11064              29              41  0.839252            1.0   
11065  11065              18              31  0.699813            1.0   
11066  11066              14              31  0.973832            1.0   

       home_team_win  home_pitcher  away_pitcher  home_team_rest  \
0                1.0             0           749       

In [9]:
from sklearn.linear_model import LinearRegression
print("LinearRegression imported successfully!")

LinearRegression imported successfully!


In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

def fill_missing_using_correlation(df):
    # Compute the correlation matrix
    corr_matrix = df.corr()

    # Iterate over each column
    for col in tqdm(df.columns):
        if df[col].isna().any():  # Only process columns with missing values
            # Find the column with the highest absolute correlation to the current column
            if corr_matrix[col].drop(col).isna().all():
                # Skip if all correlations are NaN
                continue
            corr_col = corr_matrix[col].drop(col).idxmax()

            # Drop rows where either column is NaN to fit the linear model
            valid_data = df[[col, corr_col]].dropna()
            if valid_data.empty or valid_data[corr_col].isna().all():
                # Skip if no valid data exists for linear regression
                continue

            # Fit a linear regression model
            X = valid_data[[corr_col]].values.reshape(-1, 1)  # Independent variable
            y = valid_data[col].values                        # Dependent variable
            model = LinearRegression()
            model.fit(X, y)

            # Handle missing values in the correlated column
            missing_corr_col_indices = df[df[corr_col].isna()].index
            if not missing_corr_col_indices.empty:
                # Fill missing correlated column values with its mean
                df.loc[missing_corr_col_indices, corr_col] = df[corr_col].mean()

            # Predict missing values using the linear model
            missing_indices = df[df[col].isna()].index
            df.loc[missing_indices, col] = model.predict(df.loc[missing_indices, [corr_col]].values.reshape(-1, 1))

            # Fill any remaining NaN with the column mean as fallback
            df[col] = df[col].fillna(df[col].mean())

    # Remove rows with any remaining NaN
    df = df.dropna(how='any')

    return df

# Example usage
# Assume train_data is your DataFrame with missing values
train_data = fill_missing_using_correlation(train_data)

# Display the result
print(train_data)


100%|██████████| 165/165 [00:00<00:00, 423.25it/s]

          id  home_team_abbr  away_team_abbr      date  is_night_game  \
0          0               0              30  0.698692            0.0   
1          1               1              31  0.420935            1.0   
2          2               1              32  0.434766            1.0   
3          3               2              33  0.304299            1.0   
4          4               3              34  0.034766            1.0   
...      ...             ...             ...       ...            ...   
11062  11062               5              48  0.035888            1.0   
11063  11063              11              39  0.961495            0.0   
11064  11064              29              41  0.839252            1.0   
11065  11065              18              31  0.699813            1.0   
11066  11066              14              31  0.973832            1.0   

       home_team_win  home_pitcher  away_pitcher  home_team_rest  \
0                1.0             0           749       

In [11]:
# Check if there are any NaN values in the DataFrame
has_nan = train_data.isna().any().any()

# Output the result
if has_nan:
    print("There are NaN values in train_data.")
else:
    print("No NaN values in train_data.")


No NaN values in train_data.


In [12]:
# Save the DataFrame to a CSV file
train_data.to_csv('./stage1_data/stage1_train_data_imputation_remove_row.csv', index=False)

print("train_data has been saved to train_data_imputation.csv")


train_data has been saved to train_data_imputation.csv
